# Week 1 Workshop

Order ID: Unique identifier for each order.   
Order Date: The date on which the order was placed.   
Origin Port: The port from which the shipment originates.   
Carrier: The company or service that is transporting the shipment.    
TPT: Meaning unclear   
Service Level: Meaning unclear.   
Ship ahead day count: Indicates the number of days an order was shipped ahead of schedule.   
Ship Late Day count: Indicates the number of days an order was shipped after the scheduled shipping date.   
Customer: Identifier for the customer receiving the shipment.   
Product ID: Unique identifier for the product being shipped.   
Plant Code: Identifier for the plant or location from which the product is shipped.   
Destination Port: The port to which the shipment is destined.   
Unit quantity: Number of units of the product in the shipment.   
Weight: Weight of the shipment.   

In [2]:
import pandas as pd

df = pd.read_excel('data/Supply chain logisitcs problem.xlsx')
df.head()

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.30
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.94
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.20
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.16
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.34


In [3]:
import networkx as nx

# Create an empty graph
G = nx.Graph()

# Add vertices for customers and products
customers = df['Customer'].unique()
products = df['Product ID'].unique()
G.add_nodes_from(customers)
G.add_nodes_from(products)

# Add edges for customer-product relationships
customer_product_edges = [(row['Customer'], row['Product ID']) for _, row in df.iterrows()]
G.add_edges_from(customer_product_edges)

# Add vertices for harbours
harbours = df['Origin Port'].unique()
G.add_nodes_from(harbours)

# Add directed weighted edges for harbour orders
harbour_order_edges = [(row['Origin Port'], row['Destination Port'], 1) for _, row in df.iterrows()]
G.add_weighted_edges_from(harbour_order_edges)

# Add weighted edges for product co-occurrence
product_cooccurrence_edges = []
for product1 in products:
    for product2 in products:
        if product1 != product2:
            common_customers = df[df['Product ID'].isin([product1, product2])]['Customer'].nunique()
            product_cooccurrence_edges.append((product1, product2, common_customers))
G.add_weighted_edges_from(product_cooccurrence_edges)

# Print the summary of the graph
# print(nx.info(G))

AttributeError: module 'networkx' has no attribute 'info'

In [ ]:
import networkx as nx

import matplotlib.pyplot as plt

# Create a subgraph with high degree nodes
high_degree_nodes = [node for node, degree in G.degree() if degree > 10]
subgraph = G.subgraph(high_degree_nodes)

# Set node attributes for visualization
node_colors = ['red' if node in customers else 'blue' for node in subgraph.nodes()]
node_shapes = ['o' if node in customers else 's' for node in subgraph.nodes()]
node_sizes = [100 if node in customers else 200 for node in subgraph.nodes()]

# Set edge attributes for visualization
edge_colors = ['green' if G.edges[edge]['weight'] > 1 else 'gray' for edge in subgraph.edges()]
edge_widths = [2 if G.edges[edge]['weight'] > 1 else 1 for edge in subgraph.edges()]

# Draw the network
pos = nx.spring_layout(subgraph)
nx.draw_networkx(subgraph, pos, node_color=node_colors, node_shape=node_shapes, node_size=node_sizes,
                 edge_color=edge_colors, width=edge_widths)

# Add labels
node_labels = {node: node for node in subgraph.nodes()}
nx.draw_networkx_labels(subgraph, pos, labels=node_labels)

# Show the plot
plt.show()